In [ ]:
import pandas as pd
import csv
from tqdm import tqdm

all_tweets = pd.read_excel('path to your file')


In [6]:
import openai
import json

openai.api_key = "input your api key"

def predict_GPT(tweet):
    """Analyze sentiment of a tweet using OpenAI's Chat API."""
    try:
        
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",  
            messages=[
                {
                    "role": "user",
                    "content": f'Predict the sentiment of this tweet: "{tweet}". '
                               'Return a JSON object in the format: '
                               '{"sentiment": "positive/negative/neutral", "sentiment_score": 1(if positive)/0(if neutral)/-1(if negative)}. '
                               'Only return this JSON object and nothing else. Do not include anyother contents besides Json object or I cannot extract the Json successfully.',
                }
            ],
            max_tokens=150,
        )

        # Extract raw content
        content = response['choices'][0]['message']['content']
        # print(f"Raw API Response: {content}")

        # Ensure content is stripped of whitespace
        content = content.strip()
        

        # Parse JSON response
        try:
            json_content = json.loads(content)  
        except json.JSONDecodeError as e:
            print(f"JSON decoding failed: {e}. Using eval() as fallback.")
            json_content = eval(content)  

        return [json_content['sentiment'], json_content['sentiment_score']]
    
    except Exception as e:
        print(f"Error: {e}")
        return ["na", "na"]


In [ ]:
results = []

for _, tweet in tqdm(all_tweets.iterrows(), total = len(all_tweets), desc = "Analysis Progress"):
    original = list(tweet)
    comment = tweet['content']
    re = predict_GPT(comment)

    original.extend(re)
    results.append(original)


header = list(all_tweets.columns) + ["sentiment", "sentiment_score"]
with open(f'path to your file', 'w', newline='', encoding='utf-8-sig') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(header)  
    csv_writer.writerows(results)  

print("saved successfully!")